<a href="https://colab.research.google.com/github/OladotunOlaiya/fault-detection/blob/main/fault-detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies Importation

In [ ]:
import numpy as np
from numpy.random import seed
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
#from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU,Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model,Model
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l2,l1,l1_l2
from sklearn.model_selection import RandomizedSearchCV
from keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM,Input,Dense,Dropout,RepeatVector,TimeDistributed
from keras.models import Model
from keras.regularizers import l2,l1,l1_l2


In [ ]:
seed(10)

## Importing data from various csv files 

In [ ]:
#Mounting google drive location for the dataset
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True) 

In [ ]:
file_path ='/content/gdrive/MyDrive/Colab Notebooks/Induction_motor_faults/archive/'

In [ ]:
#Bringing in the dataset into variables 
normal_files = glob.glob(file_path + "normal/**/*.csv", recursive=True)
imbalance_fault_files = glob.glob(file_path + "imbalance/**/*.csv", recursive=True)
#print the loaded csv files
print(imbalance_fault_files)



###Downsampling:



In [ ]:
#Reading in the dataset
def dataReader(files_array, output_value = 0):
    df = pd.DataFrame()
    for file in files_array:
        df_temp = pd.read_csv(file,header=None,nrows=5000)
        df = pd.concat([df,df_temp],ignore_index=True)
    df[len(df.columns)] = output_value
    return df

In [ ]:
df_normal = dataReader(normal_files, 1)

In [ ]:
df_normal 

##Data Exploration

Statistical descriptions

In [ ]:
df_normal.isnull()

In [ ]:
# Missing data in columns and rows
print(df_normal.isnull().sum(axis = 0),df_normal.isnull().sum(axis = 1))

In [ ]:
#Statistical Description
df_normal.describe()

In [ ]:
df_imbalance = dataReader(imbalance_fault_files, 0)

In [ ]:
df_imbalance

In [ ]:
df_imbalance.isnull()

In [ ]:
# Missing data in columns and rows
print(df_imbalance.isnull().sum(axis = 0),df_imbalance.isnull().sum(axis = 1))

In [ ]:
df_imbalance.describe()

In [ ]:
print (df_normal.shape,df_imbalance.shape)

Data visualization

*   correlation




In [ ]:
plt.figure(figsize=(10,10))
plt.title('normal_features')
sns.heatmap(df_normal.corr(),annot=True,fmt='.2f', square=True,cmap="Reds_r")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title('imbalance_features')
sns.heatmap(df_imbalance.corr(),annot=True,fmt='.2f', square=True,cmap="Reds_r")
plt.show()

In [ ]:
#Normal State Features
fig, axs = plt.subplots(9, sharex=False, sharey=False,figsize=(15,15))
fig.suptitle('Normal State')
for i in df_normal.columns:
    axs[i].plot(df_normal[i])
    axs[i].set_title('{} Column of Dataset'.format(i))

In [ ]:
#imbalance features
fig, axs = plt.subplots(9, sharex=False, sharey=False,figsize=(15,15))
fig.suptitle('Imbalance Fault')
for i in df_imbalance.columns:
    axs[i].plot(df_imbalance[i])
    axs[i].set_title('{} Column of Dataset'.format(i))




*   Box Plots 




In [ ]:
#Box plot for normal data before normalization
boxplot = df_normal.boxplot(rot=45, fontsize=15,figsize=(10,10))

In [ ]:
#after scaling:
scaler = MinMaxScaler()
scaled_df_normal = scaler.fit_transform(df_normal)
boxplot = pd.DataFrame(scaled_df_normal).boxplot(rot=45, fontsize=15,figsize=(10,10))

In [ ]:
#Box plot for imbalance data before normalization
boxplot = df_imbalance.boxplot(rot=45, fontsize=15,figsize=(10,10))

In [ ]:
#after scaling:
scaler = MinMaxScaler()
scaled_df_imbalance = scaler.fit_transform(df_imbalance)
boxplot = pd.DataFrame(scaled_df_imbalance).boxplot(rot=45, fontsize=15,figsize=(10,10))


## Data Preprocessing

*   Combining both normal and Imbalance data:




In [ ]:
#Join normal and imbalance data frames
df_combined = pd.concat([df_normal,df_imbalance],ignore_index=True)
df_combined.columns=['SV0','SV1','SV2','SV3','SV4','SV5','SV6','SV7','Y'] # sensor values
df_combined_no_label = df_combined.values[:, 0:-1]
df_combined.index=pd.to_datetime(df_combined.index,format='%Y-%m-%d %H:%M:%S')# assume data point was generated every 
df_combined=df_combined.sort_index()

labels =df_combined.values[:, -1]
df_combined_no_label = df_combined.values[:, 0:-1]



In [ ]:
df_combined

In [ ]:
df_combined_no_label



*   Split the separate data into train and test:




In [ ]:
Normal_data=df_combined_no_label[0:245000]# normal data 
Normal_labels=labels[0:245000]
Imablance_data=df_combined_no_label[245000:]# take 20% of the imbalance data for validation
Imbalance_labels=labels[245000:]
Normal_data.shape,Imablance_data.shape,Normal_labels.shape,Imbalance_labels.shape



 

* Split normal into train and test:    









In [ ]:
#Normal train and test
xtrain_normal,x_test_normal,ytrain_normal,ytest_normal=train_test_split(Normal_data,Normal_labels,test_size=0.2,random_state=0)
xtrain_normal.shape,ytrain_normal.shape,x_test_normal.shape,ytest_normal.shape

* Split imbalance into train and test: 

In [ ]:
#imbalance data
xtrain_imbal,x_test_imbal,ytrain_imbal,ytest_imbal=train_test_split(Imablance_data,Imbalance_labels,test_size=0.2,random_state=0)
xtrain_imbal.shape,ytrain_imbal.shape,x_test_imbal.shape,ytest_imbal.shape

* Split the combined into train and test: 

In [ ]:
#combined data
train_combined,test_combined,ytrain_combined,ytest_combined =train_test_split(df_combined_no_label,labels,test_size=0.2,random_state=20)
train_combined.shape,test_combined.shape,ytrain_combined.shape,ytest_combined.shape




*  Scaling and Reshaping the normal data into a tensor:




In [ ]:
#for the normal data:
scaler=MinMaxScaler()
x_trainN_scaled=scaler.fit_transform(xtrain_normal)

x_testN_scaled=scaler.transform(x_test_normal)
x_train_2d=x_trainN_scaled
x_trainN_scaled=x_trainN_scaled.reshape(x_trainN_scaled.shape[0],1,x_trainN_scaled.shape[1])
print('normal training data shape:',x_trainN_scaled.shape)
x_testN_scaled=x_testN_scaled.reshape(x_testN_scaled.shape[0],1,x_testN_scaled.shape[1])
print('normal testing data shape:',x_testN_scaled.shape)


*  Scaling and Reshaping the imbal data into a tensor:

In [ ]:
#for the imbalance data:
scaler=MinMaxScaler()
x_train_imbal_scaled=scaler.fit_transform(xtrain_imbal)
x_test_imbal_scaled=scaler.transform(x_test_imbal)
x_train_imbal_scaled=x_train_imbal_scaled.reshape(x_train_imbal_scaled.shape[0],1,x_train_imbal_scaled.shape[1])
print('imbalance training data shape:',x_train_imbal_scaled.shape)
x_test_imbal_scaled=x_test_imbal_scaled.reshape(x_test_imbal_scaled.shape[0],1,x_test_imbal_scaled.shape[1])
print('imbalance testing data shape:',x_test_imbal_scaled.shape)


*  Scaling and Reshaping the combined data into a tensor:

In [ ]:
#for the combined data:
scaler=MinMaxScaler()
train_combined_scaled=scaler.fit_transform(train_combined)
test_combined_scaled=scaler.transform(test_combined)
train_combined_scaled=train_combined_scaled.reshape(train_combined_scaled.shape[0],1,train_combined_scaled.shape[1])
test_combined_scaled=test_combined_scaled.reshape(test_combined_scaled.shape[0],1,test_combined_scaled.shape[1])
test_combined_scaled.shape

# LSTM autoencoder model:

In [ ]:
def LSTM_autoencoder(X=x_trainN_scaled,activation='relu',kernel_regularizer=regularizers.L2(0.00)):
    print(X.shape)
    inputs=Input(shape=(X.shape[1],X.shape[2]))
    x=inputs
    x=LSTM(16,activation='relu',return_sequences=True,kernel_regularizer=regularizers.L2(0.00))(x)
    x=LSTM(4,activation='relu',return_sequences=False)(x)
    x=RepeatVector(X.shape[1])(x)
    x=LSTM(4,activation='relu',return_sequences=True)(x)
    x=LSTM(16,activation='relu',return_sequences=True)(x)
    output=TimeDistributed(Dense(X.shape[2]))(x)
    model=Model(inputs=inputs,outputs=output)
    model.compile(optimizer='adam',loss='mae',metrics=["accuracy"])
    return model



*   Hyperparameter tuning with Randomsearch
 



In [ ]:
model=KerasClassifier(build_fn=LSTM_autoencoder,verbose=1)

kernel_regularizer=['L1','L2']
epochs=[20,50,100,150]
batch_size=[32,64,128,512]
activation=['relu','tanh']
param_grid=dict(batch_size=batch_size,activation=activation,kernel_regularizer=kernel_regularizer,epochs=epochs)
grid=RandomizedSearchCV(estimator=model,param_distributions=param_grid,n_jobs=-1,cv=3,n_iter=5)

In [ ]:
grid_result=grid.fit(x_trainN_scaled,x_train_2d)
print("Best: %f using %s" %(grid_result.best_score_,grid_result.best_params_))

## create and compile the model

In [ ]:
model=LSTM_autoencoder(x_trainN_scaled)
model.compile(optimizer='adam',loss='mae')
model.summary()

## Model Training:

In [ ]:
#training the model:
epochs_number=20
batch_size=512
history = model.fit(x_trainN_scaled,x_trainN_scaled,epochs=epochs_number,batch_size=batch_size,validation_data=(test_combined_scaled,test_combined_scaled),shuffle=True,verbose=1)

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()

In [ ]:
reconstructions = model.predict(x_trainN_scaled)
train_loss = tf.keras.losses.mae(reconstructions, x_trainN_scaled)

In [ ]:
plt.hist(train_loss[:,0], bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()



## Setting threshold




In [ ]:
threshold = np.mean(train_loss) #+ np.std(train_loss)
print("Threshold: ", threshold)

In [ ]:
def predict(model, data, threshold):
    reconstructions = model(data)
    loss = tf.keras.losses.mae(reconstructions, data)
    return tf.math.less(loss, threshold)

def print_stats(predictions, labels):
  print("Accuracy = {}".format(accuracy_score(labels, predictions)))
  #print("Precision = {}".format(precision_score(labels, predictions)))
  #print("Recall = {}".format(recall_score(labels, predictions)))

In [ ]:
def predictLabels(autoEncoderModel, inputData, threshold_val):
  recon = autoEncoderModel(inputData)
  loss_val = tf.keras.losses.mae(recon, inputData)
  predictedLabels = tf.math.less(loss_val, threshold_val)
  return predictedLabels, loss_val

def print_stats_new(predicted_val, inputDataLabels):
  print("Accuracy = {}".format(accuracy_score(inputDataLabels, predicted_val)))

## Prediction accuracy( Not the model accuracy)

In [ ]:
preds_val, loss_values = predictLabels(model, test_combined_scaled, threshold)

print_stats_new(preds_val, ytest_combined)

In [ ]:
preds_val_normal, loss_values_normal = predictLabels(model, x_trainN_scaled, threshold)

In [ ]:
print_stats_new(preds_val_normal, ytrain_normal)

In [ ]:
preds_val_imbalance, loss_values_imbalance = predictLabels(model, x_train_imbal_scaled, threshold)

In [ ]:
time = np.linspace(0,1,len(test_combined_scaled),endpoint=True)
fig, axs = plt.subplots(nrows=1, figsize=(16, 9))

#sns.lineplot(x=time, y=loss_values)
#plt.plot(loss_values_imbalance[:250], label="loss value", color='r')
#plt.plot(loss_values_normal[:250], label="normal loss value", color='g')
#plt.plot([100, threshold], label="Threshold Value")

#plt.axhline(y=threshold, label="Threshold Value", color='b')
#plt.show()

In [ ]:
preds = predict(model, test_combined_scaled, threshold)
print_stats(preds, ytest_combined)
#preds

In [ ]:
preds.shape,x_test_imbal_scaled.shape,ytest_imbal.shape

In [ ]:
#x_test_imbal_scaled.shape[0]

preds[0]

preds.numpy()

In [ ]:
#pred_scaled=pd.DataFrame(x_test_imbal_scaled[:,0]) + pd.DataFrame(ytest_imbal[:])

#pred_scaled = pd.concat([pd.DataFrame(x_test_imbal_scaled[:,0]), pd.DataFrame(ytest_imbal[:]), pd.DataFrame(preds.numpy())], axis=1, ignore_index=True)
pred_scaled = pd.concat([pd.DataFrame(test_combined_scaled[:,0]), pd.DataFrame(ytest_combined[:].astype(int)), pd.DataFrame(preds.numpy().astype(int))], axis=1, ignore_index=True)
#pred_scaled=pd.DataFrame(x_test_imbal_scaled[:,0])

#pd.DataFrame.set_index()

In [ ]:
pred_scaled

#pred_scaled

#ytest_imbal

In [ ]:
true_count = (pred_scaled[9]).value_counts()[1] 
False_count = (pred_scaled[9]).value_counts()[0]
print(true_count,False_count)

In [ ]:
#accuracy_count = (pred_scaled[8]).value_counts()

accuracy_count = (pred_scaled[pred_scaled[8] == pred_scaled[9]]).value_counts()
#true_count_2 = accuracy_count[10].value_counts()[True]
print(len(accuracy_count))

In [ ]:
p=(len(accuracy_count)/len(pred_scaled))*100
print(p)